In [ ]:
import ee
try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

In [ ]:
# Nightlights
viirs = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").select('avg_rad')

# Boundary of South Africa
sa_boundary = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq("ADM0_NAME", "South Africa"))

# Get a list of all the image IDs in the clipped collection
image_ids = viirs.aggregate_array("system:id").getInfo()
image_ids

In [ ]:
# https://gis.stackexchange.com/questions/325188/masking-pixels-inside-the-polygons-using-google-earth-engine
def maskOutside(image, geometry):
    mask = ee.Image.constant(1).clip(geometry).mask() # .not() # to mask inside
    return image.updateMask(mask)

In [ ]:
# South Africa Boundaries
minlon = 16; minlat = -35; maxlon = 34; maxlat = -21

# Define the bounding box
south_africa_bbox = ee.Geometry.Rectangle([minlon, minlat, maxlon, maxlat])

In [ ]:
# Display Nightlights South Africa
import geemap
Map = geemap.Map()
center = sa_boundary.geometry().centroid().getInfo()
Map.setCenter(center["coordinates"][0], center["coordinates"][1], 6) 
Map.addLayer(viirs.first().clip(sa_boundary), {}, "Layer")
Map.addLayer(south_africa_bbox, {}, 'Rectangle')
Map

In [ ]:
# Download each image
for image_id in image_ids:
    image = ee.Image(image_id).select('avg_rad')

    # Replace all values outside region with -9999
    image = maskOutside(image, sa_boundary).unmask(-9999)

    # Get the original CRS and geotransform of the image
    proj = image.projection().getInfo()

    # Create a filename for the downloaded image
    filename = image_id.split("/")[-1]

    # Export the image with the original CRS and geotransform
    task = ee.batch.Export.image.toDrive(
        image = image,
        region = south_africa_bbox, # sa_boundary.geometry().bounds(),
        description = filename,
        folder = "south_africa_viirs_dnb_nightlights_v1_vcmslcfg",
        crs = proj["crs"],
        crsTransform = proj["transform"],
        maxPixels = 1e13,
        fileFormat = "GeoTIFF"
    )
    task.start()

    print(f"Exporting {filename}...")

### Creating Annual Median Composites
Following: https://worldbank.github.io/OpenNightLights/tutorials/mod3_6_making_VIIRS_annual_composites.html

In [ ]:
# define our start and end years
start = 2014
end = 2022

years = ee.List.sequence(start, end)

print(f"our list has {years.size().getInfo()} years in it")

In [ ]:

colID = "NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG"
def viirs_annual_median_reduce(year):
    return ee.ImageCollection(colID).filter(
        ee.Filter.calendarRange(year,year,"year")).select("avg_rad").median().set('year', year)

# map function to each year in our list
year_comps = ee.ImageCollection.fromImages(years.map(viirs_annual_median_reduce))

In [ ]:
map2 = geemap.Map(center=center["coordinates"], zoom=6)
# map2.add_basemap('SATELLITE')
# add each layer
for year in range(start,end+1):
    img = year_comps.filterMetadata("year", "equals", year).first().clip(sa_boundary) #there's only one image, but we extract from collection
    map2.addLayer(img, {}, f"VIIRS-DNB {year}")
    # map2.addLayer(img.mask(img), {}, f"VIIRS-DNB {year}", opacity=.75)

map2.addLayerControl()
map2

In [ ]:
year_comps.aggregate_array("year").getInfo()

In [ ]:
year_comps.filterMetadata("year", "equals", year).first().projection().getInfo()

In [ ]:
proj = viirs.first().projection().getInfo()

In [ ]:
# Download each image
for year in year_comps.aggregate_array("year").getInfo():
    image = year_comps.filterMetadata("year", "equals", year).first()

    # Replace all values outside region with -9999
    image = maskOutside(image, sa_boundary).unmask(-9999)

    # Export the image with the original CRS and geotransform
    task = ee.batch.Export.image.toDrive(
        image = image,
        region = south_africa_bbox, # sa_boundary.geometry().bounds(), 
        description = str(year),
        folder = "south_africa_viirs_dnb_nightlights_v1_vcmslcfg_annual_median_composite",
        crs = proj["crs"],
        crsTransform = proj["transform"],
        maxPixels = 1e13,
        fileFormat = "GeoTIFF"
    )
    task.start()

    print(f"Exporting {year}...")